# Multimodal RAG Pipeline

## 1. Environment Setup:

This notebook implements a **minimal end-to-end Retrieval-Augmented Generation (RAG)** system
for PDF / PPT / Word documents, enhanced with Vision-Language Models (VLM).

At this stage, we focus on:
- Document parsing
- Text & image understanding
- Text embedding and vector retrieval
- Basic RAG question answering


In [1]:
#Install dependencies
!pip install -q \
  pypdf \
  python-pptx \
  python-docx \
  sentence-transformers \
  faiss-cpu \
  transformers \
  pillow \
  torch torchvision torchaudio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 11.7 MB/s eta 0:00:00


In [2]:
#Check GPU
import torch
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


CUDA available: True
GPU: Tesla T4


In [3]:
#Upload files
from google.colab import files
uploaded = files.upload()

Saving Lecture 5_IR_Websearch.pdf to Lecture 5_IR_Websearch.pdf
Saving Lecture 6 Sentimental Analysis_Short.pdf to Lecture 6 Sentimental Analysis_Short.pdf


## 2. Chunk Extraction (PDF, PPTX, DOCX)

Implement three separate pipelines for handling PDF, PPTX, and DOCX files.  
Each pipeline consists of two stages:
1. Pure text extraction  
2. Image extraction


####2.1 PDF pipeline
This pipeline extracts text and images from PDF files, organizes them into structured chunks with metadata, and prepares them for embedding and retrieval.

In [4]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 115.6 MB/s eta 0:00:00


In [5]:
from pypdf import PdfReader
import fitz  # pymupdf
from PIL import Image


def _render_pdf_page_to_pil(doc: fitz.Document, page_index0: int, zoom: float = 2.0) -> Image.Image:
    """Render one PDF page (0-based index) to PIL.Image"""
    page = doc.load_page(page_index0)
    mat = fitz.Matrix(zoom, zoom)
    pix = page.get_pixmap(matrix=mat, alpha=False)
    img = Image.frombytes("RGB", (pix.width, pix.height), pix.samples)
    return img


def _estimate_image_area_ratio(page: fitz.Page) -> float:
    """
    Estimate image area ratio on a page:
    sum(area of image blocks) / page area
    Uses PyMuPDF text/image blocks (fast & good enough for routing).
    """
    page_area = float(page.rect.width * page.rect.height) or 1.0

    img_area = 0.0
    try:
        d = page.get_text("dict")
        for b in d.get("blocks", []):
            if b.get("type") == 1:  # 1 = image block
                x0, y0, x1, y1 = b.get("bbox", (0, 0, 0, 0))
                img_area += max(0.0, float(x1 - x0)) * max(0.0, float(y1 - y0))
    except Exception:
        # fallback: if something goes wrong, return 0
        return 0.0

    ratio = img_area / page_area
    # clamp to [0,1] for safety
    return max(0.0, min(1.0, ratio))


def extract_pdf_chunks(
    path: str,
    zoom: float = 2.0,
    vlm_ratio_threshold: float = 0.10,
    keep_textless_image_pages: bool = True,
) -> list:
    """
    Extract PDF chunks with:
    - raw_text from PyPDF
    - image_area_ratio from PyMuPDF
    - page_image (PIL) only when image_area_ratio >= vlm_ratio_threshold (so VLM can work)

    Output chunk schema (keeps your fields + adds page_image):
    {
      "source_file": path,
      "type": "pdf",
      "page": 1-based,
      "raw_text": "...",
      "image_area_ratio": float,
      "route": None,
      "page_image": PIL.Image or None
    }
    """
    reader = PdfReader(path)
    doc = fitz.open(path)

    chunks = []

    n_pages = min(len(reader.pages), doc.page_count)

    for i in range(n_pages):
        page_num_1based = i + 1

        # 1.text (PyPDF)
        try:
            text = (reader.pages[i].extract_text() or "").strip()
        except Exception:
            text = ""

        # 2.image ratio (PyMuPDF)
        try:
            page = doc.load_page(i)
            image_area_ratio = _estimate_image_area_ratio(page)
        except Exception:
            image_area_ratio = 0.0

        # 3.render page image ONLY if ratio >= threshold
        page_image = None
        if image_area_ratio >= vlm_ratio_threshold:
            try:
                page_image = _render_pdf_page_to_pil(doc, i, zoom=zoom)
            except Exception:
                page_image = None

        # 4.write chunk
        # - keep textless pages if they have meaningful images and you want them retrievable by VLM caption
        if text or (keep_textless_image_pages and page_image is not None):
            chunks.append({
                "source_file": path,
                "type": "pdf",
                "page": page_num_1based,
                "raw_text": text,
                "image_area_ratio": float(image_area_ratio),
                "route": None,
                "page_image": page_image,   # <- NEW: for VLM
            })

    doc.close()
    return chunks

####2.2 PPTX pipeline

In [6]:
import io
from typing import List, Dict, Any, Optional
from PIL import Image
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE_TYPE

_pptx_cache = {}

def _get_pptx(path: str) -> Presentation:
    if path not in _pptx_cache:
        _pptx_cache[path] = Presentation(path)
    return _pptx_cache[path]


def extract_pptx_chunks(
    path: str,
    vlm_ratio_threshold: float = 0.10,   # if bigger than 0.1 then go through VLM
    max_images_per_slide: int = 3,       # max3 images per page
    keep_textless_image_slides: bool = True,
) -> List[Dict[str, Any]]:
    """
    the form of chunk keep same with pdf：
    {
      "source_file": path,
      "type": "pptx",
      "slide": 1-based,
      "raw_text": "...",
      "image_area_ratio": float,
      "route": None,
      "images": [PIL.Image, ...]   # <-- 给VLM用（PPTX版的“page_image”集合）
    }
    """
    prs = _get_pptx(path)
    chunks: List[Dict[str, Any]] = []

    # slide area
    slide_w = float(prs.slide_width or 1.0)
    slide_h = float(prs.slide_height or 1.0)
    slide_area = slide_w * slide_h if slide_w > 0 and slide_h > 0 else 1.0

    for si, slide in enumerate(prs.slides, start=1):
        parts: List[str] = []
        images_with_ratio: List[tuple] = []   # [(ratio, PIL.Image), ...]
        max_img_ratio = 0.0

        for shape in slide.shapes:
            # 1.text
            if hasattr(shape, "text") and shape.text:
                t = shape.text.strip()
                if t:
                    parts.append(t)

            # 2.pictures: ratio + extract PIL
            if shape.shape_type == MSO_SHAPE_TYPE.PICTURE:
                # 2.1ratio
                try:
                    img_area = float(shape.width * shape.height)
                    ratio = img_area / slide_area
                    if ratio > max_img_ratio:
                        max_img_ratio = ratio
                except Exception:
                    ratio = 0.0

                # 2.1only extract images if ratio is meaningful (>= threshold)
                if ratio >= vlm_ratio_threshold:
                    try:
                        blob = shape.image.blob
                        pil_img = Image.open(io.BytesIO(blob)).convert("RGB")
                        images_with_ratio.append((ratio, pil_img))
                    except Exception:
                        pass

        text = "\n".join(parts).strip()

        # only keep max_images_per_slide
        if images_with_ratio:
            images_with_ratio.sort(key=lambda x: x[0], reverse=True)
            images = [im for _, im in images_with_ratio[:max_images_per_slide]]
        else:
            images = []

        # 3.keep policy
        keep = bool(text)
        if (not keep) and keep_textless_image_slides and (max_img_ratio >= vlm_ratio_threshold):
            keep = True

        if keep:
            chunks.append({
                "source_file": path,
                "type": "pptx",
                "slide": si,
                "raw_text": text,
                "image_area_ratio": float(max_img_ratio),
                "route": None,
                "images": images,   #send image into chunk
            })

    return chunks

####2.3 DOCX pipeline

In [7]:
import io
from typing import List, Dict, Any
from PIL import Image
from docx import Document
from docx.text.paragraph import Paragraph
from docx.table import Table


def iter_block_items(parent):
    parent_elm = parent.element.body
    for child in parent_elm.iterchildren():
        if child.tag.endswith('}p'):
            yield Paragraph(child, parent)
        elif child.tag.endswith('}tbl'):
            yield Table(child, parent)


def extract_images_from_paragraph(paragraph: Paragraph) -> List[Image.Image]:
    images = []

    # run._element.xml contain drawing / blip
    # find a:blip  embed rId，retrive picture from related_parts
    try:
        for run in paragraph.runs:
            # quick filter
            if "a:blip" not in run._element.xml:
                continue

            blips = run._element.xpath(".//a:blip")
            for blip in blips:
                rId = blip.attrib.get(
                    "{http://schemas.openxmlformats.org/officeDocument/2006/relationships}embed"
                )
                if not rId:
                    continue

                image_part = paragraph.part.related_parts.get(rId)
                if image_part is None:
                    continue

                blob = image_part.blob
                try:
                    pil_img = Image.open(io.BytesIO(blob)).convert("RGB")
                    images.append(pil_img)
                except Exception:
                    pass
    except Exception:
        pass

    return images


def extract_images_from_table(table: Table) -> List[Image.Image]:
    images = []
    try:
        for row in table.rows:
            for cell in row.cells:
                for p in cell.paragraphs:
                    images.extend(extract_images_from_paragraph(p))
    except Exception:
        pass
    return images


def extract_docx_chunks(
    path: str,
    chunk_chars: int = 800,
    img_ratio_when_hit: float = 0.15,
    vlm_ratio_threshold: float = 0.10,     # ≥threshold：put images into chunk pass to VLM
    max_images_per_chunk: int = 3,
    keep_textless_image_chunks: bool = True,
) -> List[Dict[str, Any]]:

    doc = Document(path)

    chunks: List[Dict[str, Any]] = []
    buf: List[str] = []
    buf_len = 0
    buf_images: List[Image.Image] = []
    section_id = 0

    def flush():
        nonlocal buf, buf_len, buf_images, section_id

        text = "\n".join([t for t in buf if t]).strip()
        has_img = len(buf_images) > 0
        image_area_ratio = img_ratio_when_hit if has_img else 0.0


        images_for_chunk = []
        if has_img and image_area_ratio >= vlm_ratio_threshold:
            images_for_chunk = buf_images[:max_images_per_chunk]

        keep = bool(text)
        if (not keep) and keep_textless_image_chunks and has_img:
            keep = True

        if keep:
            section_id += 1
            chunks.append({
                "source_file": path,
                "type": "docx",
                "section": section_id,
                "raw_text": text,
                "image_area_ratio": float(image_area_ratio),
                "route": None,
                "images": images_for_chunk,
            })

        # reset buffers
        buf, buf_len, buf_images = [], 0, []

    # traverse the document
    for block in iter_block_items(doc):
        if isinstance(block, Paragraph):
            t = (block.text or "").strip()
            if t:
                buf.append(t)
                buf_len += len(t)

            # extract picture from paragraph
            imgs = extract_images_from_paragraph(block)
            if imgs:
                buf_images.extend(imgs)

        elif isinstance(block, Table):
            table_text_parts = []
            try:
                for row in block.rows:
                    row_text = []
                    for cell in row.cells:
                        cell_text = (cell.text or "").strip()
                        if cell_text:
                            row_text.append(cell_text)
                    if row_text:
                        table_text_parts.append(" | ".join(row_text))
            except Exception:
                pass

            if table_text_parts:
                tt = "\n".join(table_text_parts).strip()
                if tt:
                    buf.append(tt)
                    buf_len += len(tt)

            # images in table
            imgs = extract_images_from_table(block)
            if imgs:
                buf_images.extend(imgs)

        if buf_len >= chunk_chars:
            flush()

    flush()
    return chunks


####2.4 Generation
In this stage, I automatically detect the document type (PDF, PPTX, or DOCX), apply the corresponding extraction pipeline, and merge all extracted chunks into a unified list. All chunks follow a standardized schema, enabling cross-format embedding and retrieval.

In [8]:
# automatically scan document type and chunk
all_chunks = []
skipped = []

for fname in uploaded.keys():
    lower = fname.lower()
    try:
        if lower.endswith(".pdf"):
            all_chunks += extract_pdf_chunks(fname)
        elif lower.endswith(".pptx"):
            all_chunks += extract_pptx_chunks(fname)
        elif lower.endswith(".docx"):
            all_chunks += extract_docx_chunks(fname)
        else:
            skipped.append(fname)
    except Exception as e:
        print(f"Failed on {fname}: {e}")
        skipped.append(fname)

print("total chunks:", len(all_chunks))
if all_chunks:
    print("Example metadata:", {k: all_chunks[0][k] for k in all_chunks[0] if k != "raw_text"})
    print("Text preview:\n", all_chunks[0]["raw_text"][:300])

if skipped:
    print("skipped/failed files:", skipped)

total chunks: 98
Example metadata: {'source_file': 'Lecture 5_IR_Websearch.pdf', 'type': 'pdf', 'page': 1, 'image_area_ratio': 0.0, 'route': None, 'page_image': None}
Text preview:
 MH8351 Web Analytics
An Brief Introduction of
Information Retrieval and Web Search
Li Xiaoli
Nanyang Technological University


In [9]:
#make decision: VLM or text
def apply_router(chunks, vlm_ratio_threshold=0.10):
    for c in chunks:
        ratio = float(c.get("image_area_ratio", 0.0) or 0.0)
        c["route"] = "vlm" if ratio >= vlm_ratio_threshold else "direct"
    return chunks

all_chunks = apply_router(all_chunks, vlm_ratio_threshold=0.10)

from collections import Counter
print("route counts:", Counter([c["route"] for c in all_chunks]))

route counts: Counter({'direct': 71, 'vlm': 27})


##3.VLM vision language model
In this stage, I integrate a Vision-Language Model (VLM) to handle visual content that cannot be captured by pure text extraction. For each chunk that contains significant images, the corresponding page or slide is rendered into an image and passed to the VLM. The model generates a natural language caption describing the visual content (e.g., diagrams, charts, or illustrations). This caption is then appended to the original text chunk and used for downstream embedding and retrieval, enabling the system to understand and retrieve information from both textual and visual modalities.

In [10]:
import torch
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

device = "cuda" if torch.cuda.is_available() else "cpu"

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
vlm = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
vlm.eval()

@torch.inference_mode()
def caption_image(pil_img: Image.Image, max_new_tokens=50) -> str:
    inputs = processor(images=pil_img, return_tensors="pt").to(device)
    out = vlm.generate(**inputs, max_new_tokens=max_new_tokens)
    return processor.decode(out[0], skip_special_tokens=True).strip()


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [11]:
import torch
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

device = "cuda" if torch.cuda.is_available() else "cpu"

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
vlm = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
vlm.eval()

@torch.inference_mode()
def caption_image(pil_img: Image.Image, max_new_tokens=50) -> str:
    inputs = processor(images=pil_img, return_tensors="pt").to(device)
    out = vlm.generate(**inputs, max_new_tokens=max_new_tokens)
    return processor.decode(out[0], skip_special_tokens=True).strip()


In [12]:
def run_vlm_for_chunks(chunks, *, max_images_per_chunk=2):
    for c in chunks:
        if c.get("route") != "vlm":
            continue

        doc_type = c.get("type")
        cap_texts = []

        try:
            if doc_type == "pdf":

                img = c.get("page_image")
                if img is not None:
                    cap_texts.append(caption_image(img))

            elif doc_type in ("pptx", "docx"):

                imgs = c.get("images") or []
                for img in imgs[:max_images_per_chunk]:
                    cap_texts.append(caption_image(img))

            else:
                pass

        except Exception as e:
            c["image_caption"] = None
            c["vlm_error"] = str(e)
            continue

        c["image_caption"] = "\n".join([t for t in cap_texts if t]).strip() if cap_texts else None

    return chunks

all_chunks = run_vlm_for_chunks(all_chunks, max_images_per_chunk=2)


check

In [13]:
from collections import Counter
print(Counter([c["route"] for c in all_chunks]))
print("vlm captions:", sum(1 for c in all_chunks if c.get("image_caption")))


Counter({'direct': 71, 'vlm': 27})
vlm captions: 27


In [14]:
for c in all_chunks:
    if c.get("image_caption"):
        print(c["type"], c.get("page") or c.get("slide"), c["image_caption"][:200])
        break

pdf 4 a diagram of a block diagram


##4.Embedding

In [16]:
def build_embedding_text(c):
    loc = c.get("page") or c.get("slide") or c.get("section") or ""
    header = f"[{c.get('type','doc')}:{loc}]"

    parts = [header]

    raw = (c.get("raw_text") or "").strip()
    if raw:
        parts.append(raw)

    cap = (c.get("image_caption") or "").strip()
    if cap:
        parts.append("[VLM]\n" + cap)

    return "\n\n".join(parts)

for c in all_chunks:
    c["embedding_text"] = build_embedding_text(c)

print(all_chunks[0]["embedding_text"][:500])


[pdf:1]

MH8351 Web Analytics
An Brief Introduction of
Information Retrieval and Web Search
Li Xiaoli
Nanyang Technological University


##5.Sentence-Transformers

In [17]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss, json

embed_model = SentenceTransformer("all-MiniLM-L6-v2")

# 1.texts + id_map escape the empty
texts_all = [(c.get("embedding_text") or "").strip() for c in all_chunks]
id_map = [i for i,t in enumerate(texts_all) if t]
texts = [texts_all[i] for i in id_map]

# 2.encode
emb = embed_model.encode(
    texts,
    batch_size=64,
    normalize_embeddings=True,
    show_progress_bar=True
)
emb = np.asarray(emb, dtype="float32")

print("emb shape:", emb.shape)

# 3.build faiss
index = faiss.IndexFlatIP(emb.shape[1])
index.add(emb)
print("index size:", index.ntotal)

# 4.save
import json

def make_json_safe_chunk(c: dict):
#copy
    d = dict(c)

    # delete all of PIL.Image
    for k in ["images", "pil_images", "image", "page_image", "slide_images"]:
        if k in d:
            d.pop(k, None)

    for k, v in list(d.items()):
        try:
            json.dumps(v)
        except TypeError:
            d[k] = str(type(v))

    return d

safe_chunks = [make_json_safe_chunk(c) for c in all_chunks]

json.dump(
    {"id_map": id_map, "chunks": safe_chunks},
    open("store.json", "w"),
    ensure_ascii=False
)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

emb shape: (98, 384)
index size: 98


In [18]:
import faiss

index = faiss.IndexFlatIP(emb.shape[1])
index.add(emb)

print("index size:", index.ntotal)


index size: 98


####5.1 Create FAISS Index + a little test




In [19]:
def retrieve(query, topk=5):
    q = embed_model.encode([query], normalize_embeddings=True)
    q = np.array(q, dtype="float32")
    scores, ids = index.search(q, topk)

    results = []
    for s, idx in zip(scores[0], ids[0]):
        c = all_chunks[int(idx)]
        loc = c.get("page") or c.get("slide") or c.get("section")
        results.append({
            "idx": int(idx),
            "score": float(s),
            "file": c["source_file"],
            "type": c["type"],
            "loc": loc,
            "preview": (c.get("raw_text","")[:180] + "...") if c.get("raw_text") else ""
        })
    return results


In [20]:
def build_context(results, all_chunks, max_chars=1200):
    blocks = []
    for r in results:
        c = all_chunks[r["idx"]]
        loc = c.get("page") or c.get("slide") or c.get("section")
        src = f"{c['source_file']} | {c['type']} | {loc}"

        body = (c.get("embedding_text") or c.get("raw_text") or "")
        body = body[:max_chars]

        blocks.append(f"[SOURCE]\n{src}\n[CONTENT]\n{body}")
    return "\n\n---\n\n".join(blocks)


In [21]:
results = retrieve("What's the main idea of this resource？", topk=5)

for r in results:
    print(r)


{'idx': 68, 'score': 0.48919400572776794, 'file': 'Lecture 5_IR_Websearch.pdf', 'type': 'pdf', 'loc': 69, 'preview': 'Summary\n• We only give a brief introduction to IR. There are a large number of \nother topics (although Words to Vectors or Docs to Vectors are \nmore advance)\n– Statistical language...'}
{'idx': 2, 'score': 0.438303679227829, 'file': 'Lecture 5_IR_Websearch.pdf', 'type': 'pdf', 'loc': 3, 'preview': '1. Information Retrieval (IR)\n• Conceptually, IR is the study of finding needed information. \ni.e., IR helps users find information that matches their \ninformation needs. \n– Users ...'}
{'idx': 73, 'score': 0.43030574917793274, 'file': 'Lecture 6 Sentimental Analysis_Short.pdf', 'type': 'pdf', 'loc': 4, 'preview': 'Introduction – user generated content\n• Word-of-mouth on the Web\n– One can express personal experiences and opinions on almost anything, at \nreview sites, forums, discussion groups...'}
{'idx': 87, 'score': 0.41655275225639343, 'file': 'Lecture 6 Sentime

####5.2 Construct context structure for LLM

In [22]:
context = build_context(results, all_chunks)
print(context[:1000])


[SOURCE]
Lecture 5_IR_Websearch.pdf | pdf | 69
[CONTENT]
[pdf:69]

Summary
• We only give a brief introduction to IR. There are a large number of 
other topics (although Words to Vectors or Docs to Vectors are 
more advance)
– Statistical language model
– Latent semantic indexing (LSI and SVD).
• Many other interesting topics are not covered
– Web search
• Ranking: combining contents and hyperlinks
• Index compression (In order to speed up the search, tries should reside in memory. 
Index compression aims to represent the same information with fewer bytes)
– Combining multiple rankings and meta search 
70

---

[SOURCE]
Lecture 5_IR_Websearch.pdf | pdf | 3
[CONTENT]
[pdf:3]

1. Information Retrieval (IR)
• Conceptually, IR is the study of finding needed information. 
i.e., IR helps users find information that matches their 
information needs. 
– Users express their information needs as queries
• Historically, IR is about document retrieval, emphasizing 
document as the basic unit.
– Fi

#### 5.3 LLM-RAG

In [23]:
#LLM load (Qwen 0.5B)
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

gen_model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(gen_model_name)

gen_model = AutoModelForCausalLM.from_pretrained(
    gen_model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)
gen_model.eval()



tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2

In [24]:
def llm_generate(prompt, max_new_tokens=300):
    inputs = tokenizer(prompt, return_tensors="pt").to(gen_model.device)

    with torch.no_grad():
        outputs = gen_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7
        )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "Answer:" in text:
        text = text.split("Answer:", 1)[-1].strip()

    return text


In [25]:
#retrieve (FAISS)
import numpy as np

def retrieve(query: str, topk=5):
    q = embed_model.encode([query], normalize_embeddings=True)
    q = np.array(q, dtype="float32")
    scores, ids = index.search(q, topk)

    results = []
    for s, idx in zip(scores[0], ids[0]):
        idx = int(idx)
        c = all_chunks[idx]
        loc = c.get("page") or c.get("slide") or c.get("section")
        results.append({
            "idx": idx,
            "score": float(s),
            "file": c.get("source_file"),
            "type": c.get("type"),
            "loc": loc,
            "preview": (c.get("raw_text","")[:180] + "...") if c.get("raw_text") else ""
        })
    return results

In [26]:
# build_context (use embedding_text with VLM)
def build_context(results, all_chunks, max_chars_each=1200):
    blocks = []
    for r in results:
        c = all_chunks[r["idx"]]
        loc = c.get("page") or c.get("slide") or c.get("section")
        src = f"{c.get('source_file')} | {c.get('type')} | {loc}"

        body = (c.get("embedding_text") or c.get("raw_text") or "")
        body = body.strip()[:max_chars_each]

        blocks.append(f"[SOURCE]\n{src}\n[CONTENT]\n{body}")
    return "\n\n---\n\n".join(blocks)

In [27]:
# prompt template
def make_prompt(question, context):
    return f"""
You are a careful and factual assistant.

Use ONLY the context below to answer the question.
If the context is insufficient, say: "Insufficient information" and explain what is missing.
Do NOT repeat the context.
Do NOT repeat the question.
Do NOT repeat these instructions.

Answer concisely.

Context:
{context}

Question:
{question}

Answer:
"""

In [28]:
# rag_answer
def rag_answer(question: str, topk=5, max_new_tokens=300, max_chars_each=1200):
    results = retrieve(question, topk=topk)
    context = build_context(results, all_chunks, max_chars_each=max_chars_each)
    prompt = make_prompt(question, context)
    answer = llm_generate(prompt, max_new_tokens=max_new_tokens)
    return answer, results


# test
ans, hits = rag_answer("What's the main idea of these two resources？", topk=5, max_new_tokens=250)
print(ans)



The main ideas of these two resources are:

1. Lecture 5_IR_Websearch.pdf focuses on introducing the basics of Information Retrieval (IR), including concepts like proximity queries, full document queries, and information retrieval principles. It also discusses the importance of ranking and indexing for efficient search operations.

2. Lecture 5_IR_Websearch.pdf includes three examples of search engines, discussing their indexing strategies, query processing methods, ranking algorithms, and the challenges involved in maintaining such systems. These examples highlight the complexity and technical intricacies involved in building and managing an effective web search engine. 

Both resources aim to provide foundational knowledge about how search engines work and how to improve their performance through advanced techniques such as LSI, SVD, and tries. However, there are significant differences between them, reflecting the diverse nature of search engine development and the varying approache

In [29]:
ans, hits = rag_answer("What does Lecture 6 say? Please summarize.", topk=5)
print(ans)

The lecture discusses sentiment analysis and provides examples from different sources like web search results, poetry, and Wikipedia. It highlights the importance of understanding how words are used together to infer meaning and sentiments about people's opinions and emotions towards things. Additionally, the lecture uses distributional hypothesis to analyze words and their usage patterns. Finally, it demonstrates sentiment analysis using a real-world dataset and visualizations like diagrams and screenshots. Overall, the lecture aims to provide insights into human communication and emotions through language. [INSUFFICIENT INFORMATION] The lecture does not contain any specific content related to the given context. [INSUFFICIENT INFORMATION] The lecture does not mention any specific source or topic related to the given context. [INSUFFICIENT INFORMATION] The lecture does not include any detailed analysis or summaries on the given topic. [INSUFFICIENT INFORMATION] There is no provided con